In [13]:
import os
import pandas as pd
import json

from pathlib import Path

In [14]:
delays = pd.read_csv("../data/delays_on_stations.csv", index_col=0)
meteo = pd.read_csv("../data/synop_fin.csv", index_col=0)

delays["departure_time"] = pd.to_datetime(delays["departure_time"])
meteo["datetime"] = pd.to_datetime(meteo["datetime"])

display(delays.tail())
display(meteo.tail())

/Users/jakubcwynar/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,departure_time,departure_delay,latitude,longitude
13249,2018-06-06 16:45:00,17.0,53.067135,18.692424
13250,2018-06-06 18:45:00,20.0,49.878771,18.688364
13251,2018-06-06 18:45:00,19.0,49.886916,18.758581
13252,2018-06-06 18:45:00,19.0,49.892758,18.809291
13253,2018-06-06 21:15:00,19.0,54.038333,22.948804


,datetime,station_name,latitude,longitude,temperature,rainfall,wind_speed
1996060,2018-12-08 22:00:00,LESKO,49.468901,22.330382,0.50,0.0,2.00
1996061,2018-12-08 22:15:00,LESKO,49.468901,22.330382,0.49,0.0,2.01
1996062,2018-12-08 22:30:00,LESKO,49.468901,22.330382,0.55,0.0,1.85
1996063,2018-12-08 22:45:00,LESKO,49.468901,22.330382,0.69,0.0,1.51
1996064,2018-12-08 23:00:00,LESKO,49.468901,22.330382,0.90,0.0,1.00


In [15]:
DATE_FROM = "2018-06-01"
DATE_TO = "2018-06-06"

MAP_DIR = Path(os.path.abspath(""))
RAIN_IMG = MAP_DIR / "images" / "rain.png"

In [16]:
delays_range = (delays["departure_time"] >= DATE_FROM) & (delays["departure_time"] < DATE_TO)
valid_latlon = delays["latitude"] != float("inf")

meteo_range = (meteo["datetime"] >= DATE_FROM) & (meteo["datetime"] < DATE_TO)
was_rainy = (meteo["rainfall"] > 0)

delays = delays[delays_range & valid_latlon]
meteo = meteo[meteo_range & was_rainy]

In [30]:
DEFAULT_MARKER_SIZE = 0.1

def delay_to_geojson(row):
    _, data = row
    timestamp = int(data["departure_time"].timestamp()) * 1e+3
    lat, lon = data["latitude"], data["longitude"]
    radius = DEFAULT_MARKER_SIZE * data["departure_delay"]
    
    point = {
      "type": "Feature",
      "properties": {
        "times": [
          timestamp
        ],
        "iconstyle": {
          "fillColor": "red",
          "stroke": True,
          "radius": radius 
        },
        "icon": "circle"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          lon,
          lat
        ]
      }
    }
    return point
    

def rain_to_geojson(row):
    _, data = row
    timestamp = int(data["datetime"].timestamp()) * 1e+3
    lat, lon = data["latitude"], data["longitude"]
    
    rainfall = data["rainfall"]
    size = min(64, 12 * rainfall)
    icon_size = [size] * 2
    
    wind = data["wind_speed"]
    opacity = 0.3 + 0.05 * wind
    
    point = {
      "type": "Feature",
      "properties": {
        "times": [
          timestamp
        ],
        "iconstyle": {
          "iconUrl": str(RAIN_IMG),
          "iconSize": icon_size
        },
        "icon": "marker",
        "opacity": opacity
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          lon,
          lat
        ]
      }
    }
    return point

In [27]:
delays.shape

(12255, 4)

In [31]:
delay_gjson = {
  "type": "FeatureCollection",
  "features": [
    delay_to_geojson(point) for point in delays.iterrows()
  ]
}

meteo_gjson = {
  "type": "FeatureCollection",
  "features": [
    rain_to_geojson(point) for point in meteo.iterrows()
  ]
}

In [32]:
with open("delays.json", "w") as f:
    json.dump(delay_gjson, f, indent=True)
    
with open("meteo.json", "w") as f:
    json.dump(meteo_gjson, f)